# DATA 534 – API Wrapper (Implementation + Unit Tests)

This notebook runs **unit tests** for the API wrapper implementation in `api_wrapper.R`.

**How to use**
1. Make sure `api_wrapper.R` is in the same folder as this notebook.
2. Run the cells top-to-bottom.


In [ ]:
library(testthat)
library(httr2)
library(ggplot2)

# Load your implementation (must define objects/functions below)
source("api_wrapper.R")

# List of required objects/functions
required <- c(
  "author_births", "canadian_olympics",
  "supported_daily_variables", "supported_hourly_variables",
  "get_historical_weather_raw", "get_historical_weather_daily", "get_historical_weather_hourly",
  "get_supported_daily_units", "get_supported_hourly_units",
  "plot_feature"
)


In [ ]:
test_that("Required objects/functions exist in the current session", {
  missing <- required[!vapply(required, exists, logical(1), inherits = TRUE)]
  if (length(missing) > 0) {
    fail(paste0("Missing: ", paste(missing, collapse = ", ")))
  }
  succeed()
})


In [ ]:
test_that("Supported variable lists look sane", {
  expect_true(is.character(supported_daily_variables))
  expect_true(is.character(supported_hourly_variables))
  expect_gt(length(supported_daily_variables), 0)
  expect_gt(length(supported_hourly_variables), 0)
})


In [ ]:
test_that("get_historical_weather_raw returns an httr2 response (or is skipped if API unavailable)", {
  resp <- tryCatch(
    get_historical_weather_raw(
      latitude = 49.2593,
      longitude = -123.2475,
      start_date = "2020-01-01",
      end_date = "2020-01-01",
      daily_variables = supported_daily_variables[1:2],
      hourly_variables = supported_hourly_variables[1:2]
    ),
    error = function(e) e
  )

  if (inherits(resp, "error")) {
    skip(paste("API call failed:", resp$message))
  }

  expect_s3_class(resp, "httr2_response")

  # If the API is reachable, status should be 200; otherwise skip to avoid flaky failures.
  st <- tryCatch(resp_status(resp), error = function(e) NA_integer_)
  if (is.na(st)) {
    skip("Could not read HTTP status from response.")
  }
  if (st != 200) {
    skip(paste("API returned non-200 status:", st))
  }

  expect_equal(st, 200)
})


In [ ]:
test_that("get_historical_weather_daily returns a data.frame with time column", {
  df <- tryCatch(
    get_historical_weather_daily(
      latitude = 49.2593,
      longitude = -123.2475,
      start_date = "2020-01-01",
      end_date = "2020-01-03",
      daily_variables = supported_daily_variables[1:3]
    ),
    error = function(e) e
  )

  if (inherits(df, "error")) {
    skip(paste("API call failed:", df$message))
  }

  expect_s3_class(df, "data.frame")
  expect_true("time" %in% names(df))
})


In [ ]:
test_that("get_historical_weather_hourly returns a data.frame with time column", {
  df <- tryCatch(
    get_historical_weather_hourly(
      latitude = 49.2593,
      longitude = -123.2475,
      start_date = "2020-01-01",
      end_date = "2020-01-01",
      hourly_variables = supported_hourly_variables[1:3]
    ),
    error = function(e) e
  )

  if (inherits(df, "error")) {
    skip(paste("API call failed:", df$message))
  }

  expect_s3_class(df, "data.frame")
  expect_true("time" %in% names(df))
})


In [ ]:
test_that("Unit helper functions return named lists (or are skipped if API unavailable)", {
  du <- tryCatch(get_supported_daily_units(), error = function(e) e)
  if (inherits(du, "error")) skip(paste("API call failed:", du$message))
  expect_true(is.list(du))
  expect_true(length(names(du)) > 0)

  hu <- tryCatch(get_supported_hourly_units(), error = function(e) e)
  if (inherits(hu, "error")) skip(paste("API call failed:", hu$message))
  expect_true(is.list(hu))
  expect_true(length(names(hu)) > 0)
})


In [ ]:
test_that("plot_feature returns a ggplot object", {
  df <- data.frame(
    time = as.Date("2020-01-01") + 0:2,
    temperature_2m_max = c(1, 2, 3)
  )
  p <- plot_feature(df, "temperature_2m_max")
  expect_s3_class(p, "ggplot")
})


## Run all tests

If you re-run cells multiple times, testthat will print multiple 'passed' lines — that's normal.
